In [1]:
# Instalaciones
!pip install pytorch-pretrained-bert
!pip install seqeval
!pip install langdetect

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 123 kB 4.1 MB/s 
     |████████████████████████████████| 132 kB 25.0 MB/s 
     |████████████████████████████████| 79 kB 9.9 MB/s 
     |████████████████████████████████| 9.1 MB 67.8 MB/s 
     |████████████████████████████████| 140 kB 73.8 MB/s 
     |████████████████████████████████| 127 kB 48.0 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 43 kB 51 kB/s 
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16180 sha256=dc630d0b6599eb88fbd08c892a9b890c7749560748d2382e3d90644136f4a92f
  Stored in directory: /root/.cache/pip/wheels/05/96/ee/7cac4e74f3b19e3158dce26a20a1c86b353

In [2]:
# Importaciones
# GENERAL Y PREPROCESADO

import pandas as pd
import numpy as np
from tqdm import tqdm, trange
import spacy
!python -m spacy download es_core_news_sm
import es_core_news_sm
from langdetect import detect
import os.path
import datetime
# BERT
import torch
from torch.optim import Adam
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from pytorch_pretrained_bert import BertTokenizer, BertConfig
from pytorch_pretrained_bert import BertForTokenClassification, BertAdam
from seqeval.metrics import f1_score


from IPython.display import clear_output 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 12.9 MB 4.1 MB/s 
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')


In [3]:
# Conexion a drive y descompresión de los corpus
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# funciones


In [4]:
def clasifica_data(file):
  file_csv = file
  ingles = []
  espanol = []
  with open(file_csv, "r") as f:
    texto = f.read().split("\n")
  for frase in texto:
    espaciado = frase.split(" ")
    idioma = detect(frase)
    if idioma == "en":
      ingles.append(frase)
    elif idioma == "es":
      espanol.append(frase)
  return ingles, espanol

  
def data_preprocessing_es(texto, ann = None):
  nlp = es_core_news_sm.load()
  nlp_sentences = [nlp(x) for x in texto]
  list_registro = []
  for i, frase in enumerate(nlp_sentences):
    for j, token in enumerate(frase):
      list_registro.append([i, j, token.lower_, token.pos_])
  df_train = pd.DataFrame(list_registro)
  df_train.columns = ["Sentence #", "Word In Sentence #", "Word", "POS"]
  df_train = df_train[~df_train["Word"].isin([" ",""])][df_train["POS"] != "PUNCT"]
  if ann != None:
    with open(ann, "r") as f:
      texto_ann = [x.split("\t") for x in f.read().split("\n")]
    df_ann = pd.DataFrame(texto_ann)
    df_ann.columns = ["num_termino", "desc", "Word"]
    df_ann_terminos = df_ann[df_ann["num_termino"].str.contains("T")].copy()
    df_ann_terminos["TipoEntidad"] = df_ann_terminos["desc"].str.split(" ").apply(lambda x: x[0])
    df_ann_terminos["Word_list"] = df_ann_terminos["Word"].apply(lambda x: list(nlp(x)))
    df_ann_terminos = df_ann_terminos.explode("Word_list")
    df_ann_terminos["rn"] = df_ann_terminos.groupby("num_termino")["Word"].cumcount()+1
    df_ann_terminos.loc[:,"Tag"]  = "I-"+df_ann_terminos.loc[df_ann_terminos["rn"] == 1,"TipoEntidad"]
    df_ann_terminos.loc[df_ann_terminos["rn"] == 1,"Tag"] = "B-"+df_ann_terminos.loc[df_ann_terminos["rn"] == 1,"TipoEntidad"]
    df_ann_terminos = df_ann_terminos[~df_ann_terminos["Word_list"].isin([" ",""])]
    df_ann_cruce = df_ann_terminos[["num_termino","Word_list","Tag"]].drop_duplicates().reset_index(drop=True)
    df_ann_cruce.columns = ["num_termino","Word", "Tag"]
    # Algoritmo de anotacion
    # Nos quedamos con un unico registro por numero de termino, palabra y tag
    df_ann_cruce = df_ann_terminos[["num_termino","Word_list","Tag"]].drop_duplicates().reset_index(drop=True)
    df_ann_cruce.columns = ["num_termino","Word", "Tag"]
    # generamos un flag unico
    df_ann_cruce["unico"] = df_ann_cruce["num_termino"] + df_ann_cruce["Word"].\
                              apply(lambda x: str(x))
    # inicializamos el pasado a NO
    df_ann_cruce["pasado"] = "NO"
    # Inicializamos la ventana de busqueda
    ventana_busqueda = 20
    registro = []
    df_train_preproceso = df_train.copy()
    # Incializamos el ultimo termino encontrado a -1 para evitar descartar ningun termino de partida
    max_termino = -1
    # Iteramos por cada palabra del corpus y buscaremos en el dataframe de anotacion
    for num_linea, linea in enumerate(df_train_preproceso.iterrows()):
      palabra = linea[1]["Word"]
      # Definimos el dataframe de busqueda
      df_lookup = df_ann_cruce.head(ventana_busqueda)
      # Nos quedamos con el registro encontrado
      ann_match = df_lookup[df_lookup["Word"].\
                              apply(lambda x: str(x).lower()).\
                              str.replace(".","") == palabra.replace(".","")].head(1)
      # if linea[1]["Sentence #"] == 724:
      #   print(palabra)
      #   print(df_lookup)
      #   print(ann_match)
      #   print(num_linea)
      #   print(df_ann_cruce)
      if len(ann_match) > 0:
        registro.append([linea[1]["Sentence #"], linea[1]["Word"],
                      linea[1]["POS"], ann_match["Tag"].iloc[0], ann_match["num_termino"].iloc[0]])
        df_ann_cruce = df_ann_cruce.loc[df_ann_cruce["unico"] != ann_match["unico"].iloc[0],:]
        max_termino = ann_match["num_termino"].iloc[0]
        # print(len(df_ann_cruce), print(len(registro)))
      else:
        registro.append([linea[1]["Sentence #"], linea[1]["Word"],
                          linea[1]["POS"], "O", "NA"])
      if num_linea % 100 == 0 and num_linea > 0:
        df_ann_cruce.loc[df_ann_cruce["num_termino"].apply(lambda x: int(x[1:])) < int(max_termino[1:]),"pasado"] = "SI"
        df_ann_cruce = df_ann_cruce.loc[df_ann_cruce["pasado"] != "SI",:]
    df = pd.DataFrame(registro)
    df.columns = ["Sentence #", "Word", "POS", "Tag", "num_concepto"]
    data = df[["Sentence #", "Word", "POS", "Tag"]]
    MAX_LEN = int(data["Sentence #"].value_counts().quantile(0.95))
    df_tmp = (data["Sentence #"].value_counts() <= MAX_LEN)
    data = data[data["Sentence #"].isin(df_tmp[df_tmp].index.tolist())]
    return data, MAX_LEN
  else:
    return df_train[["Word", "POS"]]


def data_preprocessing_en(texto, ann = None):
  nlp = spacy.load("en_core_web_sm")
  nlp_sentences = [nlp(x) for x in texto]
  list_registro = []
  for i, frase in enumerate(nlp_sentences):
    for j, token in enumerate(frase):
      list_registro.append([i, j, token.lower_, token.pos_])
  df_train = pd.DataFrame(list_registro)
  df_train.columns = ["Sentence #", "Word In Sentence #", "Word", "POS"]
  df_train = df_train[~df_train["Word"].isin([" ",""])][df_train["POS"] != "PUNCT"]
  if ann != None:
    with open(ann, "r") as f:
      texto_ann = [x.split("\t") for x in f.read().split("\n")]
    df_ann = pd.DataFrame(texto_ann)
    df_ann.columns = ["num_termino", "desc", "Word"]
    df_ann_terminos = df_ann[df_ann["num_termino"].str.contains("T")].copy()
    df_ann_terminos["TipoEntidad"] = df_ann_terminos["desc"].str.split(" ").apply(lambda x: x[0])
    df_ann_terminos["Word_list"] = df_ann_terminos["Word"].apply(lambda x: list(nlp(x)))
    df_ann_terminos = df_ann_terminos.explode("Word_list")
    df_ann_terminos["rn"] = df_ann_terminos.groupby("num_termino")["Word"].cumcount()+1
    df_ann_terminos.loc[:,"Tag"]  = "I-"+df_ann_terminos.loc[df_ann_terminos["rn"] == 1,"TipoEntidad"]
    df_ann_terminos.loc[df_ann_terminos["rn"] == 1,"Tag"] = "B-"+df_ann_terminos.loc[df_ann_terminos["rn"] == 1,"TipoEntidad"]
    df_ann_terminos = df_ann_terminos[~df_ann_terminos["Word_list"].isin([" ",""])]
    df_ann_cruce = df_ann_terminos[["num_termino","Word_list","Tag"]].drop_duplicates().reset_index(drop=True)
    df_ann_cruce.columns = ["num_termino","Word", "Tag"]
    # Algoritmo de anotacion
    # Nos quedamos con un unico registro por numero de termino, palabra y tag
    df_ann_cruce = df_ann_terminos[["num_termino","Word_list","Tag"]].drop_duplicates().reset_index(drop=True)
    df_ann_cruce.columns = ["num_termino","Word", "Tag"]
    # generamos un flag unico
    df_ann_cruce["unico"] = df_ann_cruce["num_termino"] + df_ann_cruce["Word"].\
                              apply(lambda x: str(x))
    # inicializamos el pasado a NO
    df_ann_cruce["pasado"] = "NO"
    # Inicializamos la ventana de busqueda
    ventana_busqueda = 20
    registro = []
    df_train_preproceso = df_train.copy()
    # Incializamos el ultimo termino encontrado a -1 para evitar descartar ningun termino de partida
    max_termino = -1
    # Iteramos por cada palabra del corpus y buscaremos en el dataframe de anotacion
    for num_linea, linea in enumerate(df_train_preproceso.iterrows()):
      palabra = linea[1]["Word"]
      # Definimos el dataframe de busqueda
      df_lookup = df_ann_cruce.head(ventana_busqueda)
      # Nos quedamos con el registro encontrado
      ann_match = df_lookup[df_lookup["Word"].\
                              apply(lambda x: str(x).lower()).\
                              str.replace(".","") == palabra.replace(".","")].head(1)
      # if linea[1]["Sentence #"] == 724:
      #   print(palabra)
      #   print(df_lookup)
      #   print(ann_match)
      #   print(num_linea)
      #   print(df_ann_cruce)
      if len(ann_match) > 0:
        registro.append([linea[1]["Sentence #"], linea[1]["Word"],
                      linea[1]["POS"], ann_match["Tag"].iloc[0], ann_match["num_termino"].iloc[0]])
        df_ann_cruce = df_ann_cruce.loc[df_ann_cruce["unico"] != ann_match["unico"].iloc[0],:]
        max_termino = ann_match["num_termino"].iloc[0]
        # print(len(df_ann_cruce), print(len(registro)))
      else:
        registro.append([linea[1]["Sentence #"], linea[1]["Word"],
                          linea[1]["POS"], "O", "NA"])
      if num_linea % 100 == 0 and num_linea > 0:
        df_ann_cruce.loc[df_ann_cruce["num_termino"].apply(lambda x: int(x[1:])) < int(max_termino[1:]),"pasado"] = "SI"
        df_ann_cruce = df_ann_cruce.loc[df_ann_cruce["pasado"] != "SI",:]
    df = pd.DataFrame(registro)
    df.columns = ["Sentence #", "Word", "POS", "Tag", "num_concepto"]
    data = df[["Sentence #", "Word", "POS", "Tag"]]
    MAX_LEN = int(data["Sentence #"].value_counts().quantile(0.95))
    df_tmp = (data["Sentence #"].value_counts() <= MAX_LEN)
    data = data[data["Sentence #"].isin(df_tmp[df_tmp].index.tolist())]
    return data, MAX_LEN
  else:
    return df_train[["Word", "POS"]]


class SentenceGetter(object):
    
    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        self.empty = False
        agg_func = lambda s: [(w, p, t) for w, p, t in zip(s["Word"].values.tolist(),
                                                           s["POS"].values.tolist(),
                                                           s["Tag"].values.tolist())]
        self.grouped = self.data.groupby("Sentence #").apply(agg_func)
        self.sentences = [s for s in self.grouped]
    
    def get_next(self):
        try:
            s = self.grouped["Sentence: {}".format(self.n_sent)]
            self.n_sent += 1
            return s
        except:
            return None

def bert_formating(data, MAX_LEN, test_size=0.2, bs = 16):
  #concat sentence
  getter = SentenceGetter(data)
  word_list = [ [s[0] for s in sent] for sent in getter.sentences] 
  sentences = word_list
  labels = [[s[2] for s in sent] for sent in getter.sentences]
  tags_vals = list(set(data["Tag"].values))
  tag2idx = {t: i for i, t in enumerate(tags_vals)}
  idx2tag = {i: t for i, t in enumerate(tags_vals) }
  words = list(set(data["Word"].values))
  n_words = len(words); 
  word2idx = {w: i + 2 for i, w in enumerate(words)}
  word2idx["UNK"] = 1
  word2idx["PAD"] = 0
  idx2word = {i: w for w, i in word2idx.items()}
  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
  if device == torch.device("cuda"):
    n_gpu = torch.cuda.device_count()
    torch.cuda.get_device_name(0) 
  tokenized_texts = word_list
  tokens_ids = [[word2idx[w] for w in s] for s in tokenized_texts]
  input_ids = pad_sequences(tokens_ids,
                          maxlen=int(MAX_LEN), dtype="int64", truncating="post", padding="post")
  for i in tokens_ids:
    if len(i) > MAX_LEN:
        #print(tokens_ids)
        print("need more max_len - defect after filtering")
        MAX_LEN = len(i)
  t_list = [[tag2idx.get(l) for l in lab] for lab in labels]
  tags = pad_sequences([[tag2idx.get(l) for l in lab] for lab in labels],
                     maxlen=MAX_LEN, value=tag2idx["O"], padding="post",
                     dtype="int64", truncating="post")
  attention_masks = [[float(i>0) for i in ii] for ii in input_ids]
  #split train test
  tr_inputs, val_inputs, tr_tags, val_tags = train_test_split(input_ids, tags, 
                                                              random_state=2022, test_size=test_size)
  tr_masks, val_masks, _, _ = train_test_split(attention_masks, input_ids,
                                              random_state=2022, test_size=test_size)
  tr_inputs = torch.tensor(tr_inputs)
  val_inputs = torch.tensor(val_inputs)
  tr_tags = torch.tensor(tr_tags)
  val_tags = torch.tensor(val_tags)
  tr_masks = torch.tensor(tr_masks)
  val_masks = torch.tensor(val_masks)
  train_data = TensorDataset(tr_inputs, tr_masks, tr_tags)
  train_sampler = RandomSampler(train_data)
  train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=bs)

  valid_data = TensorDataset(val_inputs, val_masks, val_tags)
  valid_sampler = SequentialSampler(valid_data)
  valid_dataloader = DataLoader(valid_data, sampler=valid_sampler, batch_size=bs)
  return train_data, train_sampler, train_dataloader, valid_data, valid_sampler,\
            valid_dataloader, tag2idx, device, tags_vals, idx2word, val_inputs

def bert_formating_one_dataset(data, MAX_LEN, bs = 16, tag2idx = None, tags_vals=None,word2idx=None):
  #concat sentence
  getter = SentenceGetter(data)
  word_list = [ [s[0] for s in sent] for sent in getter.sentences] 
  sentences = word_list
  labels = [[s[2] for s in sent] for sent in getter.sentences]
  if tags_vals is None:
    tags_vals = list(set(data["Tag"].values))
  if tag2idx is None:
    tag2idx = {t: i for i, t in enumerate(tags_vals)}
  words = list(set(data["Word"].values))
  n_words = len(words)
  if word2idx is None:
    word2idx = {w: i + 2 for i, w in enumerate(words)}
    word2idx["UNK"] = 1
    word2idx["PAD"] = 0
  idx2word = {i: w for w, i in word2idx.items()}
  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
  if device == torch.device("cuda"):
    n_gpu = torch.cuda.device_count()
    torch.cuda.get_device_name(0) 
  tokenized_texts = word_list
  # tokens_ids = [[word2idx[w] for w in s] for s in tokenized_texts]

  tokens_ids = []
  for s in tokenized_texts:
    tmp = []
    for w in s:
      try:
        tmp.append(word2idx[w])
      except:
        tmp.append(word2idx["UNK"])
    tokens_ids.append(tmp)
  print(tokens_ids)
  input_ids = pad_sequences(tokens_ids,
                          maxlen=int(MAX_LEN), dtype="int64", truncating="post", padding="post")
  for i in tokens_ids:
    if len(i) > MAX_LEN:
        #print(tokens_ids)
        print("need more max_len - defect after filtering")
        MAX_LEN = len(i)
  t_list = [[tag2idx.get(l) for l in lab] for lab in labels]
  tags = pad_sequences([[tag2idx.get(l) for l in lab] for lab in labels],
                     maxlen=MAX_LEN, value=tag2idx["O"], padding="post",
                     dtype="int64", truncating="post")
  attention_masks = [[float(i>0) for i in ii] for ii in input_ids]
  #split train test
  tr_inputs = torch.tensor(input_ids, dtype=torch.int64)
  tr_tags = torch.tensor(tags, dtype=torch.int64)
  tr_masks = torch.tensor(attention_masks, dtype = torch.uint8)

  train_data = TensorDataset(tr_inputs, tr_masks, tr_tags)
  train_sampler = RandomSampler(train_data)
  train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=bs)
  return train_data, train_sampler, train_dataloader, tag2idx, device, tags_vals, idx2word, tr_inputs, word2idx
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=2).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

def model_creation(tag2idx, device, FULL_FINETUNING = True, reentreno= False, save_path = ""):
  model = BertForTokenClassification.from_pretrained(u"bert-base-uncased", num_labels=len(tag2idx))
  if device == torch.device("cuda"):
    model.cuda()
  if reentreno:
    print("Loading existing model...")
    model.load_state_dict(torch.load(save_path))
  if FULL_FINETUNING:
      param_optimizer = list(model.named_parameters())
      no_decay = ['bias', 'gamma', 'beta']
      optimizer_grouped_parameters = [
          {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
          'weight_decay_rate': 0.01},
          {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
          'weight_decay_rate': 0.0}
      ]
  else:
      param_optimizer = list(model.classifier.named_parameters()) 
      optimizer_grouped_parameters = [{"params": [p for n, p in param_optimizer]}]
  optimizer = Adam(optimizer_grouped_parameters, lr=3e-5)
  return model, optimizer

def training(model, optimizer, train_dataloader, valid_dataloader, tags_vals, idx2word, epochs = 200,
             max_grad_norm = 1.0, save_path = "./bert1", early_stopping=15, use_scheduler=True):
  lmbda = lambda x: 0.98 if use_scheduler else 1
  scheduler = torch.optim.lr_scheduler.MultiplicativeLR(optimizer, lr_lambda=lmbda)
  train_f1 = []
  train_losses = []
  val_losses = []
  contador_max = -1
  for _ in trange(epochs, desc="Epoch"):
      # TRAIN loop
      model.train()
      tr_loss = 0
      nb_tr_examples, nb_tr_steps = 0, 0
      for step, batch in enumerate(train_dataloader):
          # add batch to gpu
          batch = tuple(t.to(device) for t in batch)
          b_input_ids, b_input_mask, b_labels = batch
          # forward pass
          loss = model(b_input_ids, token_type_ids=None,
                      attention_mask=b_input_mask, labels=b_labels)
          # backward pass
          loss.backward()
          # track train loss
          tr_loss += loss.item()
          nb_tr_examples += b_input_ids.size(0)
          nb_tr_steps += 1
          # gradient clipping
          # torch.nn.utils.clip_grad_norm_(parameters=model.parameters(), max_norm=max_grad_norm)
          # update parameters
          optimizer.step()
          model.zero_grad()
      scheduler.step()
      # print train loss per epoch
      print("Train loss: {}".format(tr_loss/nb_tr_steps))
      # VALIDATION on validation set
      model.eval()
      eval_loss, eval_accuracy = 0, 0
      nb_eval_steps, nb_eval_examples = 0, 0
      predictions , true_labels, true_inputs = [], [],[]
      for batch in valid_dataloader:
          batch = tuple(t.to(device) for t in batch)
          b_input_ids, b_input_mask, b_labels = batch          
          with torch.no_grad():
              tmp_eval_loss = model(b_input_ids, token_type_ids=None,
                                    attention_mask=b_input_mask, labels=b_labels)
              logits = model(b_input_ids, token_type_ids=None,
                            attention_mask=b_input_mask)
          logits = logits.detach().cpu().numpy()
          label_ids = b_labels.to('cpu').numpy()
          inputs = b_input_ids.to('cpu').numpy()
          
          true_inputs.append(inputs)
          predictions.extend([list(p) for p in np.argmax(logits, axis=2)])
          true_labels.append(label_ids)
          
          tmp_eval_accuracy = flat_accuracy(logits, label_ids)
          
          eval_loss += tmp_eval_loss.mean().item()
          eval_accuracy += tmp_eval_accuracy
          
          nb_eval_examples += b_input_ids.size(0)
          nb_eval_steps += 1
      eval_loss = eval_loss/nb_eval_steps
      print("Validation loss: {}".format(eval_loss))
      print("Validation Accuracy: {}".format(eval_accuracy/nb_eval_steps))
      pred_tags = [tags_vals[p_i] for p in predictions for p_i in p]
      valid_tags = [tags_vals[l_ii] for l in true_labels for l_i in l for l_ii in l_i]
      valid_inputs = [[idx2word[l_ii] for l_ii in l_i] for l in  true_inputs  for l_i in l ]
      f1 = f1_score([pred_tags], [valid_tags])
      train_f1.append(f1)
      train_losses.append(tr_loss/nb_tr_steps)
      val_losses.append(eval_loss)

      max_f1 = max(train_f1)
      if f1 == max_f1:
        contador_max = 1
        torch.save(model.state_dict(), save_path)
      if contador_max > 0:
        contador_max += 1
      print("F1-Score: " + str(train_f1[-1]))
      if round(max_f1, 2) > 0.1 and contador_max > early_stopping :
        print("Early stopping...")
        return 0
def training_scheduler(model, optimizer, loss_fn, train_dataloader, valid_dataloader, tags_vals, idx2word, epochs=200,
             max_grad_norm=1.0, save_path="./bert1", early_stopping=15, print_freq=10, use_scheduler=True):
    lmbda = lambda x: 0.98 if use_scheduler else 1
    scheduler = torch.optim.lr_scheduler.MultiplicativeLR(optimizer, lr_lambda=lmbda)

    train_f1 = []
    train_losses = []
    val_losses = []
    contador_max = -1

    for ep in range(epochs):
        # TRAIN loop
        model.train()
        tr_loss = 0
        nb_tr_examples, nb_tr_steps = 0, 0
        for step, batch in enumerate(train_dataloader):
            # add batch to gpu
            batch = tuple(t.to(device) for t in batch)
            b_input_ids, b_input_mask, b_labels = batch
            # forward pass
            logits = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
            loss = model.compute_loss(loss_fn, logits, b_labels, b_input_mask)
            # backward pass
            loss.backward()
            # track train loss
            tr_loss += loss.item()
            nb_tr_examples += b_input_ids.size(0)
            nb_tr_steps += 1
            # gradient clipping
            # torch.nn.utils.clip_grad_norm_(parameters=model.parameters(), max_norm=max_grad_norm)
            # update parameters
            optimizer.step()
            model.zero_grad()
        scheduler.step()
        # print train loss per epoch
        if ep % print_freq == 0:
            print(f"{ep}/{epochs}: Train loss: {tr_loss / nb_tr_steps}")

        # VALIDATION on validation set
        model.eval()
        eval_loss, eval_accuracy = 0, 0
        nb_eval_steps, nb_eval_examples = 0, 0
        predictions, true_labels, true_inputs = [], [], []
        for batch in valid_dataloader:
            batch = tuple(t.to(device) for t in batch)
            b_input_ids, b_input_mask, b_labels = batch
            with torch.no_grad():
                logits = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)

            tmp_eval_loss = model.compute_loss(loss_fn, logits, b_labels, b_input_mask)

            logits = logits.cpu().numpy()
            label_ids = b_labels.to('cpu').numpy()
            inputs = b_input_ids.to('cpu').numpy()

            true_inputs.append(inputs)
            predictions.extend([list(p) for p in np.argmax(logits, axis=2)])
            true_labels.append(label_ids)

            tmp_eval_accuracy = flat_accuracy(logits, label_ids)

            eval_loss += tmp_eval_loss.item()
            eval_accuracy += tmp_eval_accuracy

            nb_eval_examples += b_input_ids.size(0)
            nb_eval_steps += 1

        eval_loss = eval_loss / nb_eval_steps
        if ep % print_freq == 0:
            print("\tValidation loss: {}".format(eval_loss))
            print("\tValidation Accuracy: {}".format(eval_accuracy / nb_eval_steps))
        pred_tags = [tags_vals[p_i] for p in predictions for p_i in p]
        valid_tags = [tags_vals[l_ii] for l in true_labels for l_i in l for l_ii in l_i]
        valid_inputs = [[idx2word[l_ii] for l_ii in l_i] for l in true_inputs for l_i in l]
        f1 = f1_score([pred_tags], [valid_tags])
        train_f1.append(f1)
        train_losses.append(tr_loss / nb_tr_steps)
        val_losses.append(eval_loss)

        max_f1 = max(train_f1)
        if f1 == max_f1:
            contador_max = 1
            torch.save(model.state_dict(), save_path)
        if contador_max > 0:
            contador_max += 1
        if ep % print_freq == 0:
            print(f"\tF1-Score: {train_f1[-1]}\n")

        if round(max_f1, 2) > 0.1 and contador_max > early_stopping:
            print("We should apply early stopping now")

def evaluate(model, valid_dataloader, tag2idx, device, tags_vals, idx2word, val_inputs, save_path = "./bert1",
             guarda_resultado="/content/drive/MyDrive/COLAB - TFM/resultado_entrenamiento1.csv"):
  #evaluate model
  model = BertForTokenClassification.from_pretrained(u"bert-base-uncased", num_labels=len(tag2idx))
  model.load_state_dict(torch.load(save_path))
  if device == torch.device("cuda"):
    model.cuda()  
  model.eval()
  predictions = []
  true_labels = []
  true_inputs = []

  eval_loss, eval_accuracy = 0, 0
  nb_eval_steps, nb_eval_examples = 0, 0
  print(len(valid_dataloader))
  for batch in tqdm(valid_dataloader):
      #print(len(batch))
      batch = tuple(t.to(device) for t in batch)
      b_input_ids, b_input_mask, b_labels = batch

      with torch.no_grad():
          tmp_eval_loss = model(b_input_ids, token_type_ids=None,
                                attention_mask=b_input_mask, labels=b_labels)
          logits = model(b_input_ids, token_type_ids=None,
                        attention_mask=b_input_mask)
          
      logits = logits.detach().cpu().numpy()
      predictions.extend([list(p) for p in np.argmax(logits, axis=2)])
      label_ids = b_labels.to('cpu').numpy()
      inputs = b_input_ids.to('cpu').numpy()
      true_inputs.append(inputs)
      
      
      true_labels.append(label_ids)
      tmp_eval_accuracy = flat_accuracy(logits, label_ids)

      eval_loss += tmp_eval_loss.mean().item()
      eval_accuracy += tmp_eval_accuracy

      nb_eval_examples += b_input_ids.size(0)
      nb_eval_steps += 1

  pred_tags = [[tags_vals[p_i] for p_i in p] for p in predictions]
  valid_tags = [[tags_vals[l_ii] for l_ii in l_i] for l in true_labels for l_i in l ]
  valid_inputs = [[idx2word[l_ii] for l_ii in l_i] for l in  true_inputs  for l_i in l ]

  count = 0 
  all_data = 0
  registro_resultado = []
  for i,j,ll in zip(pred_tags,valid_tags,val_inputs):
      for k,l,kk in zip(i,j,ll):
        count += 1
        registro_resultado.append([k,l,idx2word[kk.item()]])
        all_data += 1
  df_alldata = pd.DataFrame(registro_resultado)
  df_alldata.columns = ["pred_tags", "valid_tags", "val_inputs"]
  df_alldata[df_alldata["val_inputs"] != "PAD"].\
        to_csv(guarda_resultado,sep=";", encoding = "utf-8")
  return pred_tags, valid_tags, valid_inputs

# No funciones

In [5]:
import pandas as pd
import matplotlib.pyplot as plt
from math import floor, ceil
from sklearn.model_selection import train_test_split

In [6]:

raiz = "/content/drive/MyDrive/COLAB - TFM/"
tag2idx = {'B-Reference': 0, 'I-Concept': 1, 'O': 2, 'I-Predicate': 3, 'B-Action': 4, 'B-Predicate': 5, 'B-Concept': 6, 'I-Action': 7}
tags_vals = ['B-Reference', 'I-Concept', 'O', 'I-Predicate', 'B-Action', 'B-Predicate', 'B-Concept', 'I-Action']
device = "cuda"
modelo_ES25 = raiz + "1200_bert_es25_sum_freq_pos_iterMODELO_FINAL"
model_ES25, optimizer_ES50 = model_creation(tag2idx, device, FULL_FINETUNING = True, reentreno= True, save_path = modelo_ES25)
modelo_ES50 = raiz + "1200_bert_es50_sum_freq_pos_iterMODELO_FINAL"
model_ES50, optimizer_ES50 = model_creation(tag2idx, device, FULL_FINETUNING = True, reentreno= True, save_path = modelo_ES50)
modelo_EN25 = raiz + "1200_bert_en25_sum_freq_pos_iterMODELO_FINAL"
model_EN25, optimizer_EN25 = model_creation(tag2idx, device, FULL_FINETUNING = True, reentreno= True, save_path = modelo_EN25)
modelo_EN50 = raiz + "1200_bert_en50_sum_freq_pos_iterMODELO_FINAL"
model_EN50, optimizer_EN50 = model_creation(tag2idx, device, FULL_FINETUNING = True, reentreno= True, save_path = modelo_EN50)

100%|██████████| 407873900/407873900 [00:27<00:00, 14631727.42B/s]


Loading existing model...
Loading existing model...
Loading existing model...
Loading existing model...


In [7]:
num_reg = [1200]
raiz = "/content/drive/MyDrive/COLAB - TFM/"
df_cord = pd.read_csv(raiz+"cord_resumen_oraciones.csv", index_col=0)
df_wikinews = pd.read_csv(raiz+"wikinews_resumen_oraciones.csv", index_col=0)
df_medline = pd.read_csv(raiz+"medline_resumen_oraciones.csv", index_col=0)

# seleccionamos los archivos que conforman el conjunto de test

In [26]:
num_reg = [1200]
reg = 1200
raiz = "/content/drive/MyDrive/COLAB - TFM/"
df_cord = pd.read_csv(raiz+"cord_resumen_oraciones.csv", index_col=0)
df_wikinews = pd.read_csv(raiz+"wikinews_resumen_oraciones.csv", index_col=0)
df_medline = pd.read_csv(raiz+"medline_resumen_oraciones.csv", index_col=0)


corpus_anotacion = (["/content/corpora-master/2021/ref/training/wikinews.300.es.txt", "/content/corpora-master/2021/ref/training/wikinews.300.es.ann"],
                    ["/content/corpora-master/2021/ref/training/medline.1200.es.txt", "/content/corpora-master/2021/ref/training/medline.1200.es.ann"],
                    ["/content/corpora-master/2021/ref/testing/cord.150.txt", "/content/corpora-master/2021/ref/testing/cord.150.ann"],
                    ["/content/corpora-master/2021/ref/testing/medline.75.txt", "/content/corpora-master/2021/ref/testing/medline.75.ann"],
                    ["/content/corpora-master/2021/ref/testing/wikinews.75.txt", "/content/corpora-master/2021/ref/testing/wikinews.75.ann"],
                    ["/content/corpora-master/2021/ref/develop/cord.50.txt", "/content/corpora-master/2021/ref/develop/cord.50.ann"],
                    ["/content/corpora-master/2021/ref/develop/medline.25.txt", "/content/corpora-master/2021/ref/develop/medline.25.ann"],
                    ["/content/corpora-master/2021/ref/develop/wikinews.25.txt", "/content/corpora-master/2021/ref/develop/wikinews.25.ann"],
                    )

In [53]:
df_cord_dev = df_cord.loc[df_cord["fichero"] == 150,:]
df_cord_dev.loc[:, "nombre_fichero"] = "cord"
df_wikinews_dev = df_wikinews.loc[df_wikinews["fichero"] == 75,:]
df_wikinews_dev.loc[:, "nombre_fichero"] = "wikinews"
df_medline_dev = df_medline.loc[df_medline["fichero"] == 75,:]
df_medline_dev.loc[:, "nombre_fichero"] = "medline"

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1667: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value


In [54]:
df_dev = pd.concat([df_cord_dev, df_wikinews_dev, df_medline_dev], axis=0).reset_index(drop=True)
df_train = pd.read_csv(raiz+str(reg)+"_sum_freq_pos.csv")

In [55]:
df_train.head()

,Unnamed: 0,Sentence #,fichero,nombre_fichero
0,0,17,50,cord
1,1,29,50,cord
2,2,10,50,cord
3,3,44,50,cord
4,4,0,50,cord


In [57]:
df_dev.head()

,Sentence #,longitud,sum_freq_palabra,rank_freq_palabra,sum_freq_pos,rank_freq_pos,dif_wv_33,fichero,nombre_fichero
0,0,27,2,126,6,72,0.085513,150,cord
1,1,21,5,66,2,138,0.105187,150,cord
2,2,23,5,71,5,98,-0.099989,150,cord
3,3,21,2,114,5,87,-0.146534,150,cord
4,4,23,4,96,4,102,0.049033,150,cord


6122
2398


In [64]:
modelo_ES25

'/content/drive/MyDrive/COLAB - TFM/1200_bert_es25_sum_freq_pos_iterMODELO_FINAL'

In [ ]:
modelo_ES25 = raiz + "1200_bert_es25_sum_freq_pos_iterMODELO_FINAL"
modelo_ES50 = raiz + "1200_bert_es50_sum_freq_pos_iterMODELO_FINAL"
modelo_EN25 = raiz + "1200_bert_en25_sum_freq_pos_iterMODELO_FINAL"
modelo_EN50 = raiz + "1200_bert_en50_sum_freq_pos_iterMODELO_FINAL"

In [73]:
modelo_ES25 = raiz + "120000_bert_es25_sum_freq_pos_iterBASELINE"
modelo_ES50 = raiz + "120000_bert_es50_sum_freq_pos_iterBASELINE"
modelo_EN25 = raiz + "120000_bert_en25_sum_freq_pos_iterBASELINE"
modelo_EN50 = raiz + "120000_bert_en50_sum_freq_pos_iterBASELINE"

In [77]:
# !! cambiar
reg = 120000
df_cord_dev = df_cord.loc[df_cord["fichero"] == 150,:]
df_cord_dev.loc[:, "nombre_fichero"] = "cord"
df_wikinews_dev = df_wikinews.loc[df_wikinews["fichero"] == 75,:]
df_wikinews_dev.loc[:, "nombre_fichero"] = "wikinews"
df_medline_dev = df_medline.loc[df_medline["fichero"] == 75,:]
df_medline_dev.loc[:, "nombre_fichero"] = "medline"
df_dev = pd.concat([df_cord_dev, df_wikinews_dev, df_medline_dev], axis=0).reset_index(drop=True)
df_train = pd.read_csv(raiz+str(reg)+"_sum_freq_pos.csv")
sufijo = "_preprocessing.csv"
archivos = ["wikinews.25.txt","wikinews.75.txt","wikinews.300.es.txt",
            "medline.25.txt","medline.75.txt","medline.1200.es.txt",
            "cord.50.txt","cord.150.txt"]
for i, archivo in enumerate(archivos):
  if i == 0:
    df = pd.read_csv(raiz+archivo+sufijo, index_col=0)
    df["fichero"] = archivo.split(".")[1]
    df["nombre_fichero"] = archivo.split(".")[0]
  else:
    df_tmp = pd.read_csv(raiz+archivo+sufijo, index_col=0)
    df_tmp["fichero"] = archivo.split(".")[1]
    df_tmp["nombre_fichero"] = archivo.split(".")[0]
    df = pd.concat([df,df_tmp], axis=0).reset_index(drop=True)
# !! cambiar
lim_i = 25
lim_f = 50
df_dev_seleccion =df_dev
df["fichero"] = df["fichero"].astype(int)
df_dev = pd.merge(df, df_dev_seleccion[["Sentence #", "fichero", "nombre_fichero"]], on=["Sentence #", "fichero", "nombre_fichero"], how="inner")
print(len(df_dev))
df_dev = df_dev.loc[(df_dev["longitud"] > lim_i) & (df_dev["longitud"] <= lim_f), :].copy()
df_dev["Sentence #"] = df_dev["Sentence #"].astype(str) + "_" + df_dev["fichero"].astype(str)  + "_" + df_dev["nombre_fichero"]
print(len(df_dev))
df_ingles_dev = df_dev.loc[df_dev["nombre_fichero"] == "cord"]
df_espanol_dev = df_dev.loc[df_dev["nombre_fichero"] != "cord"]
df_seleccion = df_train
df["fichero"] = df["fichero"].astype(int)
df_train = pd.merge(df, df_seleccion, on=["Sentence #", "fichero", "nombre_fichero"], how="inner")
print(len(df_train))
df_train = df_train.loc[(df_train["longitud"] > lim_i) & (df_train["longitud"] <= lim_f), :].copy()
df_train["Sentence #"] = df_train["Sentence #"].astype(str) + "_" + df_train["fichero"].astype(str) + "_" + df_train["nombre_fichero"]
print(len(df_train))
df_ingles_analiza = df_train.loc[df_train["nombre_fichero"] == "cord"]
print(len(df_ingles_analiza))
df_espanol_analiza = df_train.loc[df_train["nombre_fichero"] != "cord"]
print(len(df_espanol_analiza))
# !! cambiar
data = df_espanol_analiza.loc[:,["Sentence #", "Word", "POS", "Tag"]]
# data = df_ingles_analiza.loc[:,["Sentence #", "Word", "POS", "Tag"]]
tag2idx = {'B-Reference': 0, 'I-Concept': 1, 'O': 2, 'I-Predicate': 3, 'B-Action': 4, 'B-Predicate': 5, 'B-Concept': 6, 'I-Action': 7}
tags_vals = ['B-Reference', 'I-Concept', 'O', 'I-Predicate', 'B-Action', 'B-Predicate', 'B-Concept', 'I-Action']
MAX_LEN = lim_f
BS = 16
train_data, train_sampler, train_dataloader, tag2idx_tr, device, tags_vals_tr, idx2word_tr, tr_inputs, word2idx = bert_formating_one_dataset(data, MAX_LEN, tag2idx = tag2idx, tags_vals=tags_vals, bs = BS)
# !! cambiar
data_dev = df_espanol_dev.loc[:,["Sentence #", "Word", "POS", "Tag"]]
# data_dev = df_ingles_dev.loc[:,["Sentence #", "Word", "POS", "Tag"]]
# !! cambiar
modelo = modelo_ES50
resultado = "baseline_es50.csv"

BS = len(data_dev)
valid_data, valid_sampler, valid_dataloader, tag2idx, device, tags_vals, idx2word, val_inputs, word2idx = bert_formating_one_dataset(data_dev, MAX_LEN, tag2idx = tag2idx, tags_vals=tags_vals, bs = BS,word2idx=word2idx)
pred_tags, valid_tags, valid_inputs = evaluate(modelo, valid_dataloader, tag2idx, device, tags_vals, idx2word, val_inputs, save_path = modelo,
            guarda_resultado=raiz+resultado)


/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1667: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value


6122
2398
23140
5377
603
4774
[[1318, 1283, 689, 1386, 1318, 204, 877, 778, 166, 1154, 190, 705, 1158, 1419, 914, 246, 400, 1318, 101, 627, 877, 945, 1246, 897, 628, 166, 358, 987, 1349, 1386, 897, 274], [897, 547, 705, 340, 166, 320, 1318, 163, 665, 275, 1318, 245, 705, 812, 1065, 1028, 166, 1048, 772, 849, 868, 358, 325, 1303, 299, 166, 972], [1030, 1210, 1424, 839, 550, 1318, 80, 1424, 407, 1442, 87, 166, 897, 1329, 1039, 275, 897, 1279, 166, 294, 877, 1450, 1065, 560, 330, 275, 1412, 1036, 1417, 1419, 406, 122, 1022, 592, 166, 364, 166, 1032, 275, 57], [1450, 1319, 1454, 1318, 252, 1145, 320, 854, 613, 1018, 764, 735, 74, 1438, 1054, 229, 877, 358, 1217, 1424, 676, 651, 1386, 585, 897, 461, 877, 204, 275, 897, 1027, 1374, 1116], [897, 1436, 1018, 166, 149, 891, 1424, 716, 911, 383, 897, 224, 985, 362, 1145, 1318, 1226, 1386, 667, 275, 897, 257, 1103, 1374, 149, 317], [788, 897, 671, 166, 358, 438, 967, 275, 1001, 1175, 166, 1122, 1318, 870, 1052, 166, 804, 1245, 336, 566, 166, 897,

100%|██████████| 1/1 [00:00<00:00,  8.31it/s]


In [ ]:
# HASTA AQUÍ

In [ ]:

df_cord_dev["nombre_fichero"] = "cord"
df_wikinews_dev["nombre_fichero"] = "wikinews"
df_medline_dev["nombre_fichero"] = "medline"
N_ITERACIONES = 5
df_dev[["Sentence #", "fichero", "nombre_fichero"]].to_csv(raiz+"dataset_dev.csv")
# 
for iteracion in ["MODELO_FINAL"]:
  for reg in num_reg:  
    lim_i_vw = floor(reg/3/2)
    lim_f_vw = ceil(reg/3/2)
    lim_i = floor(reg/3)
    lim_f = ceil(reg/3)
    print(reg,lim_i_vw,  lim_f_vw, lim_i, lim_f)

    ### dif_wv_33
    df_cord_wv_1 = df_cord.sort_values("dif_wv_33", ascending=False).head(lim_i_vw)[["Sentence #", "fichero"]]
    df_cord_wv_2 = df_cord.sort_values("dif_wv_33", ascending=True).tail(lim_f_vw)[["Sentence #", "fichero"]]
    df_cord_wv = pd.concat([df_cord_wv_1, df_cord_wv_2], axis=0).reset_index(drop=True)
    df_cord_wv["nombre_fichero"] = "cord"

    df_medline_wv_1 = df_medline.sort_values("dif_wv_33", ascending=False).head(lim_f_vw)[["Sentence #", "fichero"]]
    df_medline_wv_2 = df_medline.sort_values("dif_wv_33", ascending=True).tail(lim_f_vw)[["Sentence #", "fichero"]]
    df_medline_wv = pd.concat([df_medline_wv_1, df_medline_wv_2], axis=0).reset_index(drop=True)
    df_medline_wv["nombre_fichero"] = "medline"

    df_wikinews_wv_1 = df_wikinews.sort_values("dif_wv_33", ascending=False).head(lim_i_vw)[["Sentence #", "fichero"]]
    df_wikinews_wv_2 = df_wikinews.sort_values("dif_wv_33", ascending=True).tail(lim_f_vw)[["Sentence #", "fichero"]]
    df_wikinews_wv = pd.concat([df_wikinews_wv_1, df_wikinews_wv_2], axis=0).reset_index(drop=True)
    df_wikinews_wv["nombre_fichero"] = "wikinews"
    dataset_wv = pd.concat([df_cord_wv, df_medline_wv, df_wikinews_wv], axis=0).drop_duplicates().reset_index(drop=True)
    dataset_wv.to_csv(raiz+str(reg)+"_dif_wv_33.csv")

    ### sum_freq_palabra
    df_cord_wv = df_cord.sort_values("sum_freq_palabra", ascending=False).head(lim_i)[["Sentence #", "fichero"]]
    df_cord_wv["nombre_fichero"] = "cord"

    df_medline_wv = df_medline.sort_values("sum_freq_palabra", ascending=False).head(lim_f)[["Sentence #", "fichero"]]
    df_medline_wv["nombre_fichero"] = "medline"

    df_wikinews_wv = df_wikinews.sort_values("sum_freq_palabra", ascending=False).head(lim_i)[["Sentence #", "fichero"]]
    df_wikinews_wv["nombre_fichero"] = "wikinews"
    dataset_freq_palabra = pd.concat([df_cord_wv, df_medline_wv, df_wikinews_wv], axis=0).drop_duplicates().reset_index(drop=True)
    dataset_freq_palabra.to_csv(raiz+str(reg)+"_sum_freq_palabra.csv")

    ### sum_freq_pos
    df_cord_wv = df_cord.sort_values("sum_freq_pos", ascending=False).head(lim_i)[["Sentence #", "fichero"]]
    df_cord_wv["nombre_fichero"] = "cord"

    df_medline_wv = df_medline.sort_values("sum_freq_pos", ascending=False).head(lim_f)[["Sentence #", "fichero"]]
    df_medline_wv["nombre_fichero"] = "medline"

    df_wikinews_wv = df_wikinews.sort_values("sum_freq_pos", ascending=False).head(lim_i)[["Sentence #", "fichero"]]
    df_wikinews_wv["nombre_fichero"] = "wikinews"
    dataset_freq_pos = pd.concat([df_cord_wv, df_medline_wv, df_wikinews_wv], axis=0).drop_duplicates().reset_index(drop=True)
    dataset_freq_pos.to_csv(raiz+str(reg)+"_sum_freq_pos.csv")

    ### random
    df_cord_wv = df_cord.sample(min(lim_i, len(df_cord) ))[["Sentence #", "fichero"]]
    df_cord_wv["nombre_fichero"] = "cord"

    df_medline_wv = df_medline.sample(min(lim_i, len(df_medline_wv) ))[["Sentence #", "fichero"]]
    df_medline_wv["nombre_fichero"] = "medline"

    df_wikinews_wv = df_wikinews.sample(min(lim_i, len(df_wikinews_wv) ))[["Sentence #", "fichero"]]
    df_wikinews_wv["nombre_fichero"] = "wikinews"
    dataset_random = pd.concat([df_cord_wv, df_medline_wv, df_wikinews_wv], axis=0).drop_duplicates().reset_index(drop=True)
    dataset_random.to_csv(raiz+str(reg)+"_random.csv")

    print(len(dataset_wv), len(dataset_freq_palabra), len(dataset_freq_pos), len(dataset_random))

    sufijo = "_preprocessing.csv"
    archivos = ["wikinews.25.txt","wikinews.75.txt","wikinews.300.es.txt",
                "medline.75.txt","medline.1200.es.txt",
                "cord.150.txt"]
    for i, archivo in enumerate(archivos):
      if i == 0:
        df = pd.read_csv(raiz+archivo+sufijo, index_col=0)
        df["fichero"] = archivo.split(".")[1]
        df["nombre_fichero"] = archivo.split(".")[0]
      else:
        df_tmp = pd.read_csv(raiz+archivo+sufijo, index_col=0)
        df_tmp["fichero"] = archivo.split(".")[1]
        df_tmp["nombre_fichero"] = archivo.split(".")[0]
        df = pd.concat([df,df_tmp], axis=0).reset_index(drop=True)
    # lim_i, lim_f = (-1, 25)
    lim_i, lim_f = (25, 50)

    df_dev_seleccion = pd.read_csv(raiz+"dataset_dev.csv", index_col=0)
    df["fichero"] = df["fichero"].astype(int)
    df_dev = pd.merge(df, df_dev_seleccion, on=["Sentence #", "fichero", "nombre_fichero"], how="inner")
    print(len(df_dev))
    df_dev = df_dev.loc[(df_dev["longitud"] > lim_i) & (df_dev["longitud"] <= lim_f), :].copy()
    df_dev["Sentence #"] = df_dev["Sentence #"].astype(str) + "_" + df_dev["fichero"].astype(str)  + "_" + df_dev["nombre_fichero"]
    print(len(df_dev))
    df_ingles_dev = df_dev.loc[df_dev["nombre_fichero"] == "cord"]
    df_espanol_dev = df_dev.loc[df_dev["nombre_fichero"] != "cord"]
    # Seleccionar las oraciones que forman cada dataframe. '_random.csv'
    sufijos = ['_sum_freq_pos.csv']
    BS = 32
    NUM_EPOCHS = 450
    for suf in sufijos:
      num = reg
      resultado = "resultado_"+str(num)+suf
      df_seleccion = pd.read_csv(raiz+str(num)+suf, index_col=0)
      df["fichero"] = df["fichero"].astype(int)
      df_train = pd.merge(df, df_seleccion, on=["Sentence #", "fichero", "nombre_fichero"], how="inner")
      print(len(df_train))
      df_train = df_train.loc[(df_train["longitud"] > lim_i) & (df_train["longitud"] <= lim_f), :].copy()
      df_train["Sentence #"] = df_train["Sentence #"].astype(str) + "_" + df_train["fichero"].astype(str) + "_" + df_train["nombre_fichero"]
      print(len(df_train))
      df_ingles_analiza = df_train.loc[df_train["nombre_fichero"] == "cord"]
      df_espanol_analiza = df_train.loc[df_train["nombre_fichero"] != "cord"]

      # Entrenar con los distintos dataframes
      modelo_en = str(num) + "_bert_en" + str(lim_f) + suf.split(".")[0] + "_iter"+str(iteracion)
      modelo_es = str(num) + "_bert_es" + str(lim_f)  + suf.split(".")[0] + "_iter"+str(iteracion)
      raiz = "/content/drive/MyDrive/COLAB - TFM/"
      log_file = "/content/drive/MyDrive/COLAB - TFM/full_training"+ suf 
      df_cruce_ingles = df_ingles_analiza
      df_cruce_espanol = df_espanol_analiza
      for data, modelo, MAX_LEN, lan, data_dev in ([df_cruce_espanol.loc[:,["Sentence #", "Word", "POS", "Tag"]], raiz + modelo_es, lim_f, "es", df_espanol_dev.loc[:,["Sentence #", "Word", "POS", "Tag"]]],                                
                                    [df_cruce_ingles.loc[:,["Sentence #", "Word", "POS", "Tag"]],raiz+modelo_en, lim_f, "en", df_ingles_dev.loc[:,["Sentence #", "Word", "POS", "Tag"]]]):
        with open(log_file, "a+") as f:
          f.write(";".join([str(datetime.datetime.now()), suf, modelo, "\n"]))
        if len(data) > 0:
          print(modelo)
          print(len(data))
          tag2idx = {'B-Reference': 0, 'I-Concept': 1, 'O': 2, 'I-Predicate': 3, 'B-Action': 4, 'B-Predicate': 5, 'B-Concept': 6, 'I-Action': 7}
          tags_vals = ['B-Reference', 'I-Concept', 'O', 'I-Predicate', 'B-Action', 'B-Predicate', 'B-Concept', 'I-Action']
          train_data, train_sampler, train_dataloader, tag2idx_tr, device, tags_vals_tr, idx2word_tr, tr_inputs, word2idx = bert_formating_one_dataset(data, MAX_LEN, tag2idx = tag2idx, tags_vals=tags_vals, bs = BS)
          print(tag2idx, tags_vals)
          valid_data, valid_sampler, valid_dataloader, tag2idx, device, tags_vals, idx2word, val_inputs, word2idx = bert_formating_one_dataset(data_dev, MAX_LEN, tag2idx = tag2idx, tags_vals=tags_vals, bs = BS,word2idx=word2idx)
          print(tag2idx, tags_vals)
          if os.path.exists(modelo):
            with open(log_file, "a+") as f:
              f.write("Escojo modelo existente: " +  modelo +"\n")
            model, optimizer = model_creation(tag2idx, device, FULL_FINETUNING = True, reentreno= False, save_path = modelo)
          else:
            model, optimizer = model_creation(tag2idx, device, FULL_FINETUNING = True)
          training(model, optimizer,train_dataloader, valid_dataloader, tags_vals, idx2word, epochs = NUM_EPOCHS,
                      max_grad_norm = 1.0, save_path = modelo, early_stopping=NUM_EPOCHS)
          pred_tags, valid_tags, valid_inputs = evaluate(model, valid_dataloader, tag2idx, device, tags_vals, idx2word, val_inputs, save_path = modelo,
                      guarda_resultado=raiz+resultado+"_validacion_"+str(MAX_LEN) + "_iter"+str(iteracion)+"_"+lan+".csv")
          pred_tags, valid_tags, valid_inputs = evaluate(model, train_dataloader, tag2idx_tr, device, tags_vals_tr, idx2word_tr, tr_inputs, save_path = modelo,
                      guarda_resultado=raiz+resultado+"_entrenamiento_"+str(MAX_LEN) + "_iter"+str(iteracion)+"_"+lan+".csv")
          os.system("zip "+modelo)
        else:
          print("No hay datos")
      clear_output()